<img src="../../assets/mimocko_logo.png" style="margin-left:auto;margin-right:auto;width:50%">

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import datetime

In [ ]:
path_to_file = '../../../data'

# Viaggi

In [ ]:
viaggi = pd.read_csv(
    f'{path_to_file}/viaggi.csv',
    sep='*',
    decimal=','
)
viaggi 
# si nota:
#  - presenza di NaN
#  - presenza di colonne con dentro liste
#  - presenza di colonne con formati di tempo da interpretare
#  - presenza di colonna con unità di misura/percentuale

## Funzioni utili

In [ ]:
# un paio di funzioni che ci torneranno comoda per gestire 
# timestampRitiro, timestampConsegna
# luogoRitiro, luogoConsegna

def mean_of_timestamps(timestamp_list):
    sum_of_timestamp = 0
    for timestamp in timestamp_list:
        sum_of_timestamp = sum_of_timestamp+timestamp.timestamp()
    return pd.Timestamp.fromtimestamp(sum_of_timestamp/len(timestamp_list))

def process_viaggi_data(viaggi):
    
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # conversione timestampRitiro, timestampConsegna a lista di stringhe
        viaggi["timestampRitiro"] = viaggi["timestampRitiro"].apply(lambda location_list: eval(location_list))
        viaggi["timestampConsegna"] = viaggi["timestampConsegna"].apply(lambda location_list: eval(location_list))

        # conversione timestampRitiro e timestampConsegna a lista di datetime
        viaggi["timestampRitiro"] = viaggi["timestampRitiro"].apply(lambda timestr_list: [pd.to_datetime(timestr) for timestr in timestr_list])
        viaggi["timestampConsegna"] = viaggi["timestampConsegna"].apply(lambda timestr_list: [pd.to_datetime(timestr) for timestr in timestr_list])
        
        # un altro possibile stimatore puntuale per timestamp e coordinate: il valor medio

        # aggregazione (media) dei timestamp
        viaggi["timestampRitiro"] = viaggi["timestampRitiro"].apply(mean_of_timestamps)
        viaggi["timestampConsegna"] = viaggi["timestampConsegna"].apply(mean_of_timestamps)

        # conversione luogoRitiro, luogoConsegna a lista di float
        viaggi["luogoConsegna"] = viaggi["luogoConsegna"].apply(lambda location_list: eval(location_list))
        viaggi["luogoRitiro"] = viaggi["luogoRitiro"].apply(lambda location_list: eval(location_list))

        # aggregazione (media) dei luoghi
        viaggi["luogoConsegna"] = viaggi["luogoConsegna"].apply(lambda coord_list: np.mean(coord_list, axis=0))
        viaggi["luogoRitiro"] = viaggi["luogoRitiro"].apply(lambda coord_list: np.mean(coord_list, axis=0))
    
    return viaggi    

In [ ]:
viaggi = process_viaggi_data(viaggi)

## Data processing

### conversione tempoViaggio a timedelta

### conversione distanzaPercorsa a float

### conversione caricaBatteriaInizio, caricaBatteriaFine a float


### fill dei vuoti nel campo `dueCaschiDisponibili` con valore testuale

# Utenti

In [ ]:
# leggo utenti
utenti = pd.read_csv(
    f'{path_to_file}/utenti.csv',
    sep='|',
    decimal=',',
    dtype={
        'LAT INDIRIZZO': float,
        'LON INDIRIZZO': float,
        'CONSENSO al TRATTAMENTO dei DATI': bool
    },
    parse_dates=[
        'DATA di NASCITA',
        'DATA ULTIMO RINNOVO PATENTE'
    ]
)
utenti

# Scooter

In [ ]:
scooter = pd.read_csv(
    f'{path_to_file}/scooter.csv',
    sep=';',
)
scooter 
# si nota:
#  - presenza di colonna con unità di misura/percentuale
#  - presenza di colonna con data


## Conversione del costo a float


## Quante "versioni" di veicoli ci sono in giro?

### numero di veicolo per versione

### merge viaggi e scooter


In [ ]:
viaggi_scooter = ...

### numero di viaggi per versione

In [ ]:
# numero di viaggi per versione, andamendo nel tempo --> ERRORE
viaggi_scooter.groupby("Versione n°")["timestampRitiro"].hist(bins=20,
                                                              alpha=.5,
                                                              legend=True,
                                                              xrot=90)

In [ ]:
# numero di viaggi per versione, andamendo nel tempo --> ERRORE
viaggi_scooter.groupby("Versione n°")["timestampRitiro"].hist(bins=pd.date_range('2018-10-01',
                                                                                 '2021-04-01',
                                                                                 freq='M'),alpha=.5,legend=True, xrot=90)

## Calcolare il contachilometri complessivo per ogni veicolo

### km totali per veicolo

### plot distribuzione

## Ci sono utilizzi dei veicoli con patente scaduta?

### merge viaggi e utenti

### Una possibiità: cercare utenti che hanno fatto viaggi con patente non rinnovata da più di dieci anni


## Il profilo di utilizzo orario varia lungo la settimana?

### Estrazione giorno della settimana e ora dei viaggi

### calcolo numero di viaggi per giorno/ora

### Es.: plot dell'andamento giornaliero di lunedì

### Es.: plot dell'andamento giornaliero di domenica